In [1]:
import psycopg2
import pandas as pd
import csv
import DB_CONFIG

### Create connection and cursor

In [2]:
# conn = psycopg2.connect(host="localhost", port = 5432, database="aio_enem", user="nicolagp", password="123")
conn = psycopg2.connect(host=DB_CONFIG.host,
                            port = DB_CONFIG.port,
                            database=DB_CONFIG.database,
                            user=DB_CONFIG.user,
                            password=DB_CONFIG.password)
                            
# Create a cursor object
cur = conn.cursor()

In [58]:
def get_students(n, conn):
    q = f"""
    SELECT id
    FROM enem_student
    ORDER BY RANDOM()
    LIMIT {n};
    """
    return list(pd.read_sql_query(q, conn)["id"].values)
students = get_students(10, conn)

In [44]:
def get_question_ids(area, conn):
    # get sub exams that are on a given area
    q = """
    SELECT id
    FROM enem_subexam
    WHERE area = 'CN'
    """
    se_ids = tuple(pd.read_sql_query(q, conn).values.flatten())

    # get question ids from area
    q = f"""
        SELECT DISTINCT(inep_code)
        FROM enem_subexamquestion
            JOIN enem_question
            ON enem_subexamquestion.question_id = enem_question.id
        WHERE sub_exam_id IN {se_ids}
        """
    return pd.read_sql_query(q, conn).values.flatten()

In [48]:
inep_questions = get_question_ids("MT", conn)

In [72]:
def get_answers(inep_code, conn):
    q = f"""
    SELECT inep_code, correct, skill_id
    FROM enem_question
    WHERE inep_code IN {tuple(inep_questions)}
    """
    return pd.read_sql_query(q, conn).set_index("inep_code")
answers = get_answers(inep_questions, conn)
answers

,correct,skill_id
inep_code,,
6584,D,6
6781,D,20
7217,E,24
7222,B,24
7507,E,20
...,...,...
97540,D,29
97711,D,1
97740,E,10


In [84]:
def get_question_student(students, inep_questions, conn):
    """
    Returns a dataframe with questions answered by a student
    Needs a student id and a connection to the db
    """
    q = f"""
    SELECT student_id, letter, inep_code, skill_id
    FROM enem_questionstudent
        JOIN enem_question
        ON enem_question.id = enem_questionstudent.question_id
    WHERE student_id IN {tuple(students)} AND inep_code IN {tuple(inep_questions)}
    ORDER BY student_id ASC, RANDOM();
    """
    return pd.read_sql_query(q, conn)
stu = get_question_student(students, inep_questions, conn)
stu

,student_id,letter,inep_code,skill_id
0,1366864,B,18231,14
1,1366864,B,76032,23
2,1366864,E,8757,19
3,1366864,A,23921,7
4,1366864,C,81780,18
...,...,...,...,...
445,9740488,C,25580,28
446,9740488,E,10883,3
447,9740488,A,83854,4
448,9740488,D,27106,1


In [85]:
def correct(x, answers):
    """
    given that a 'questions' answer key is defined, it checks wether a student's answer is correct
    """
    return 1 if x[1] == answers.loc[x[2], "correct"] else 0

In [86]:
stu["correct"] = stu.apply(correct, axis=1, args=(answers,))
stu

,student_id,letter,inep_code,skill_id,correct
0,1366864,B,18231,14,0
1,1366864,B,76032,23,1
2,1366864,E,8757,19,1
3,1366864,A,23921,7,1
4,1366864,C,81780,18,1
...,...,...,...,...,...
445,9740488,C,25580,28,0
446,9740488,E,10883,3,1
447,9740488,A,83854,4,1
448,9740488,D,27106,1,1


In [92]:
answers.loc[25580]

correct      D
skill_id    28
Name: 25580, dtype: object